# Práctica 2: SparkSQL TAREA 1

El objetivo será también agregar usando como clave primaria la terna usuario (User), modelo (Model) y movimiento ejecutado (gt).
En concreto, hay que crear un RDD (por cada fichero). A partir de cada RDD, obtendremos un DataFrame infiriendo el esquema.
A partir de los DataFrames iniciales, obtendremos un registro por cada usuario, modelo y clase con la media, desviación estándar y valor máximo y mínimo de la secuencia del movimiento ejecutado.
Una vez hecho esto, se deberá concatenar mediante join los registros de giróscopo y acelerómetro de los relojes por un lado y de los teléfonos por otro. Finalmente se creará un DataFrame único (mediante union) con los DataFrames de teléfonos y relojes.

In [5]:
# imports necesarios
from pyspark.sql import Row
from pyspark.sql import functions as F

# funciones 
# Función que crea el DF de cada fichero cargado, con las columnas que nos interesan de los datos de partida.
def crearDataframe(elem) :
    parts_elem = elem.map(lambda l: l.split(","))
    parts_elem = parts_elem.map(lambda p: Row(user=p[6], model=p[7], gt=p[9], x=p[3], y=p[4], z=p[5]))
    df = spark.createDataFrame(parts_elem)
    
    return df


# Función que para cada DF creado realiza los cálculos de máximo, mínimo, media y desviación de las variables x,
# y, z, agrupando por las columnas user, model y gt. Devuelve un DF con todos los cálculos requeridos.
def calculosDataframe(df_inicial) :
    df_max = df_inicial.groupBy('user', 'model', 'gt').agg(F.max('x'), F.max('y'), F.max('z'))
    df_min = df_inicial.groupBy('user', 'model', 'gt').agg(F.min('x'), F.min('y'), F.min('z'))
    df_avg = df_inicial.groupBy('user', 'model', 'gt').agg(F.avg('x'), F.avg('y'), F.avg('z'))
    df_std = df_inicial.groupBy('user', 'model', 'gt').agg(F.stddev('x'), F.stddev('y'), F.stddev('z'))

    df_calculos = df_max.join(df_min, on=['user', 'model', 'gt'], how='inner')\
             .join(df_avg, on=['user', 'model', 'gt'], how='inner')\
             .join(df_std, on=['user', 'model', 'gt'], how='inner') 

    return df_calculos


# Función que recibe los 4 DFs de cálculos, primero hace un join para los dos DFs de watch y luego otro join para
# los dos DFs de movil, para finalmente unir ambos DFs y obtener sólo un DF que contenga todos los registros calculados.
def unirDataframes(df_watch_acc, df_watch_giros, df_movil_acc, df_movil_giros) :
    df_watch = df_watch_acc.join(df_watch_giros, on=['user', 'model', 'gt'], how='full')
    df_movil = df_movil_acc.join(df_movil_giros, on=['user', 'model', 'gt'], how='full')
    
    df_final = df_watch.unionAll(df_movil)
    
    return df_final


In [10]:
# Cargamos los ficheros de datos. Luego sobre cada uno creamos un DF, que pasamos a la función que realiza los
# cálculos para tener el DF de cálculos de cada fichero. Finalmente utilizamos la función que hace los join y union
# y acabamos obteniendo el DF final.

watch_acc_g = sc.textFile('datos/Watch_accelerometer.csv')
watch_giros_g = sc.textFile('datos/Watch_gyroscope.csv')
movil_acc_g = sc.textFile('datos/Phones_accelerometer.csv')
movil_giros_g = sc.textFile('datos/Phones_gyroscope.csv')

parts_w_acc_g = crearDataframe(watch_acc_g)
parts_w_giros_g = crearDataframe(watch_giros_g)
parts_m_acc_g = crearDataframe(movil_acc_g)
parts_m_giros_g = crearDataframe(movil_giros_g)

df_watch_acc_g = calculosDataframe(parts_w_acc_g)
df_watch_giros_g = calculosDataframe(parts_w_giros_g)
df_movil_acc_g = calculosDataframe(parts_m_acc_g)
df_movil_giros_g = calculosDataframe(parts_m_giros_g)

df_final = unirDataframes(df_watch_acc_g, df_watch_giros_g, df_movil_acc_g, df_movil_giros_g)

df_final.show()


+----+-------+----------+---------+---------+---------+-------------+--------------+-------------+-------------------+-------------------+--------------------+------------------+------------------+-------------------+---------+-------------+---------+--------------+--------------+--------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|user|  model|        gt|   max(x)|   max(y)|   max(z)|       min(x)|        min(y)|       min(z)|             avg(x)|             avg(y)|              avg(z)|    stddev_samp(x)|    stddev_samp(y)|     stddev_samp(z)|   max(x)|       max(y)|   max(z)|        min(x)|        min(y)|        min(z)|              avg(x)|              avg(y)|              avg(z)|     stddev_samp(x)|     stddev_samp(y)|     stddev_samp(z)|
+----+-------+----------+---------+---------+---------+-------------+--------------+-------------+-------------------+-------------------+--------------------+-